# Analysis of Layout ID

## Relevant documents

- (Python Client Repo)[https://github.com/Green-Fusion/energy-management-backend/tree/main/python_client]
- (Klemmenbelegung)[https://docs.google.com/spreadsheets/d/1nkdkx2rI6nVKgoKBgkCUtfwEwuv8kptrRUXcXtfv0NM/edit?gid=247168398#gid=247168398]
- (Hypothesis for Klemmenbelegung)[https://docs.google.com/spreadsheets/d/1TSTxMCgEvuoayzOfx1MUqlV0tiqsVTBRN8aldlnFXxA/edit?gid=0#gid=0]

In [ ]:
from energy_management_client import BackendPythonClient

In [ ]:
def get_buildings(client, customers):
    try:
        buildings = []
        for c in customers:
            client.configure(customer_id=c.id)
            c_buildings = client.buildings.get_buildings()
            buildings.extend(c_buildings)

        return buildings
    except Exception as e:
        print("API Call Failed:", e)




In [ ]:
if __name__ == "__main__":
    # init the client
    client = BackendPythonClient(
        backend_endpoint="https://dev.green-fusion.de/services/energy-management-backend/v1/api",
    )

    client.login(
        auth_endpoint="https://auth.green-fusion.de",
        realm="development",
        client_id="backend_python_client", # backend_python_client, datascience_development_api
        grant_type="device_code", #"client_credentials",  # or "password", "device_code"
    )

    customers = client.customers.get_list_of_customers()
    buildings = client.buildings.get_buildings()
    print(buildings)